In [2]:
from matplotlib.pylab import *
from jax import  numpy as jnp
from flax import linen as nn
import numpy as np
from typing import Any, Callable, Optional, Tuple, Type, List
from jax import lax, random, numpy as jnp
import einops
import torchio
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import jax
# import monai_swin_nD
import tensorflow as tf
# import monai_einops
import torch 
import einops
import torchio as tio
import optax
from flax.training import train_state  # Useful dataclass to keep train state
from torch.utils.data import DataLoader
import h5py
import jax
from jax.config import config
from skimage.segmentation import mark_boundaries
import cv2
import functools
# from torch.utils.tensorboard import SummaryWriter
# import torchvision.transforms.functional as F
# import torchvision
import flax.jax_utils as jax_utils
import tensorflow as tf
from jax_smi import initialise_tracking
import ml_collections
import time
import more_itertools
import toolz
from subprocess import Popen
from flax.training import checkpoints, train_state
from flax import struct, serialization
import orbax.checkpoint
from datetime import datetime
from flax.training import orbax_utils
from flax.core.frozen_dict import freeze
import flax
import pandas as pd
# from ..super_voxels.SIN.SIN_jax_2D_with_gratings.model_sin_jax_2D import SpixelNet
# from ..super_voxels.SIN.SIN_jax_2D_with_gratings.model_sin_jax_utils_2D import *
# from ..super_voxels.SIN.SIN_jax_2D_with_gratings.shape_reshape_functions import *


In [2]:
base_size=(4,4)
a = np.ones(base_size)
a= np.pad(a,((2,2),(2,2)))
a[1,4]=1
a[6,4]=1
a[4,1]=1
a[4,6]=1
# a= jnp.array(a)
pd.DataFrame(a.astype(int))

,0,1,2,3,4,5,6,7
0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,1,1,1,1,0,0
3,0,0,1,1,1,1,0,0
4,0,1,1,1,1,1,1,0
5,0,0,1,1,1,1,0,0
6,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0


In [5]:
def differentiable_eq_simpl(a:float,b:float):
    return a*b+(1-a)*(1-b)

def differentiable_and_simple(a:float,b:float):
    return a*b

def differentiable_or_simple(a:float,b:float):
    return a*b+(1-a)*a+(1-b)*a

def translate_in_axis(mask:jnp.ndarray, axis:int,is_forward:int,translation_val:int,mask_shape:Tuple[int]):
    mask= jnp.take(mask, indices=jnp.arange(translation_val*(1-is_forward),mask_shape[axis]-translation_val* is_forward),axis=axis )
    to_pad=np.array([[0,0],[0,0]])
    is_back=1-is_forward
    to_pad[axis,is_back]=translation_val
    return  jnp.pad(mask,to_pad)

def simple_erode(prev,curr,next):
    return differentiable_or_simple(prev*curr,curr*next)

v_soft_erode_around=jax.vmap(simple_erode, in_axes=(0,0,0), out_axes=0)
v_v_soft_erode_around=jax.vmap(simple_erode, in_axes=(0,0,0), out_axes=0)

mask_shape= a.shape

a_b=translate_in_axis(a, axis=0,is_forward=0,translation_val=1,mask_shape=mask_shape)
a_f=translate_in_axis(a, axis=0,is_forward=1,translation_val=1,mask_shape=mask_shape)

a_u=translate_in_axis(a, axis=1,is_forward=0,translation_val=1,mask_shape=mask_shape)
a_d=translate_in_axis(a, axis=1,is_forward=1,translation_val=1,mask_shape=mask_shape)

zero_erode=v_v_soft_erode_around(a_b,a,a_f)*v_v_soft_erode_around(a_u,a,a_d) 
print(f"not eroded \n{pd.DataFrame(a.astype(int))} \n zero_erode \n{pd.DataFrame(zero_erode.astype(int))}")


# def soft_erode(prev,curr,next):
#     """ 
#     will take the previous current and next point 
#     and will do modified differentiable binary erosion
#     need to take into account at least 3 pixels at a time and if it is 010 do not change otherwise optimal solutions will be thin strips so 110 should become 100 
#     110 100 
#     011 001
#     010 010
#     111 111
#     000 000
#     """
#     edge_diffrent= 1-differentiable_eq_simpl(prev,next)
#     should_be_0=differentiable_and_simple(edge_diffrent,curr)
#     return (1-should_be_0)*curr





# print(f"sh 0 {soft_erode(1,1,0)}  sh 0 {soft_erode(1,1,0)}  sh 0 {soft_erode(0,0,1)}  sh 0 {soft_erode(0,0,0)} sh 1 {soft_erode(0,1,0)} sh 1 {soft_erode(1,1,1)}    ")


not eroded 
   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  1  0  0  0
2  0  0  1  1  1  1  0  0
3  0  0  1  1  1  1  0  0
4  0  1  1  1  1  1  1  0
5  0  0  1  1  1  1  0  0
6  0  0  0  0  1  0  0  0
7  0  0  0  0  0  0  0  0 
 zero_erode 
   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0  0
2  0  0  1  1  1  0  0  0
3  0  0  1  1  1  0  0  0
4  0  0  1  1  1  1  0  0
5  0  0  0  0  1  0  0  0
6  0  0  0  0  0  0  0  0
7  0  0  0  0  0  0  0  0


In [4]:
def soft_erode_around(prev,curr,next,up,down):
    """ 
    will take the previous current and next point 
    and will do modified differentiable binary erosion
    """
    edge_diffrent= 1-differentiable_eq_simpl(prev,next)
    edge_diffrent_up_down= 1-differentiable_eq_simpl(up,down)
    should_be_0_a=differentiable_and_simple(edge_diffrent,curr)
    should_be_0_b=differentiable_and_simple(edge_diffrent_up_down,curr)
    should_be_0=differentiable_or_simple(should_be_0_a,should_be_0_b)
    return (1-should_be_0)*curr

In [5]:
v_soft_erode_around=jax.vmap(soft_erode_around, in_axes=(0,0,0,0,0), out_axes=0)
v_v_soft_erode_around=jax.vmap(v_soft_erode_around, in_axes=(0,0,0,0,0), out_axes=0)
mask_shape=a.shape
a_b=translate_in_axis(a, axis=0,is_forward=0,translation_val=1,mask_shape=mask_shape)
a_f=translate_in_axis(a, axis=0,is_forward=1,translation_val=1,mask_shape=mask_shape)

a_u=translate_in_axis(a, axis=1,is_forward=0,translation_val=1,mask_shape=mask_shape)
a_d=translate_in_axis(a, axis=1,is_forward=1,translation_val=1,mask_shape=mask_shape)

zero_erode=v_v_soft_erode_around(a_b,a,a_f,a_u,a_d)
print(f"not eroded \n{pd.DataFrame(a.astype(int))} \n zero_erode \n{pd.DataFrame(zero_erode.astype(int))}")


not eroded 
   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  1  0  0  0
2  0  0  1  1  1  1  0  0
3  0  0  1  1  1  1  0  0
4  0  1  1  1  1  1  1  0
5  0  0  1  1  1  1  0  0
6  0  0  0  0  1  0  0  0
7  0  0  0  0  0  0  0  0 
 zero_erode 
   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0  0
2  0  0  0  0  1  0  0  0
3  0  0  1  1  1  1  0  0
4  0  1  1  1  1  1  1  0
5  0  0  0  0  1  0  0  0
6  0  0  0  0  0  0  0  0
7  0  0  0  0  0  0  0  0


In [6]:
a_b=translate_in_axis(zero_erode, axis=0,is_forward=0,translation_val=1,mask_shape=mask_shape)
a_f=translate_in_axis(zero_erode, axis=0,is_forward=1,translation_val=1,mask_shape=mask_shape)
a_u=translate_in_axis(zero_erode, axis=1,is_forward=0,translation_val=1,mask_shape=mask_shape)
a_d=translate_in_axis(zero_erode, axis=1,is_forward=1,translation_val=1,mask_shape=mask_shape)
zero_erode_2=v_v_soft_erode_around(a_b,zero_erode,a_f,a_u,a_d)

print(f"not eroded \n{pd.DataFrame(a.astype(int))} \n zero_erode_2 \n{pd.DataFrame(zero_erode_2.astype(int))}")


not eroded 
   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  1  0  0  0
2  0  0  1  1  1  1  0  0
3  0  0  1  1  1  1  0  0
4  0  1  1  1  1  1  1  0
5  0  0  1  1  1  1  0  0
6  0  0  0  0  1  0  0  0
7  0  0  0  0  0  0  0  0 
 zero_erode_2 
   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0  0
2  0  0  0  0  0  0  0  0
3  0  0  0  0  1  0  0  0
4  0  1  0  0  1  0  1  0
5  0  0  0  0  0  0  0  0
6  0  0  0  0  0  0  0  0
7  0  0  0  0  0  0  0  0


In [7]:
# v_soft_erode=jax.vmap(soft_erode, in_axes=(0,0,0), out_axes=0)
# v_v_soft_erode=jax.vmap(v_soft_erode, in_axes=(0,0,0), out_axes=0)
# mask_shape=a.shape
# a_b=translate_in_axis(a, axis=0,is_forward=0,translation_val=1,mask_shape=mask_shape)
# a_f=translate_in_axis(a, axis=0,is_forward=1,translation_val=1,mask_shape=mask_shape)
# zero_erode=v_v_soft_erode(a_b,a,a_f)
# a_b=translate_in_axis(a, axis=1,is_forward=0,translation_val=1,mask_shape=mask_shape)
# a_f=translate_in_axis(a, axis=1,is_forward=1,translation_val=1,mask_shape=mask_shape)
# one_erode=v_v_soft_erode(a_b,a,a_f)
# res= zero_erode*one_erode

# print(f"not eroded \n{pd.DataFrame(a.astype(int))} \n zero_erode \n{pd.DataFrame(zero_erode.astype(int))}")

In [8]:
# print(f"one_erode \n{pd.DataFrame(one_erode.astype(int))}   \n eroded \n{pd.DataFrame(res.astype(int))}  ")

In [66]:
from itertools import product
from itertools import starmap
r_x_total= 3
r_y_total= 3
true_num_dim=3
aa=list(product(range(1,r_x_total+1),range(1,r_y_total+1),range(true_num_dim)))
# list(map(lambda el: ,aa))
aa
bb=list(map(lambda tupl: (tupl[1],tupl[0]) ,list(enumerate(aa)) ))
# dd=dict(list(zip(aa,bb)))
dd=dict(bb)
dd[(1,2,0)]

def get_index(r_x,r_y,dim_stride):
    r_x=r_x-1
    r_y=r_y-1
    return (r_x*r_y_total*true_num_dim) + (r_y*true_num_dim) + dim_stride

def get_fun(res):
    return res 
aab=list(map(lambda tupl: functools.partial(get_fun,res=(tupl,1, (2,0,7.1)))  ,aa))
aab[1]()

def selesctt(index,v):
    return jax.lax.switch(index, aab)

In [24]:
from itertools import product
from itertools import starmap
#r_x,r_y,dim_stride
gold=[
    [1,0,0]
    ,[1,1,1]
    ,[2,1,0]
    ,[2,2,1]
    ,[3,2,0]
    ,[3,3,1]
    ]
r_x_total= 3
r_y_total= 3
true_num_dim=2
# aa=list(product(range(1,r_x_total+1),range(1,r_y_total+1),range(true_num_dim)))
aa=list(product(range(1,r_x_total+1),range(0,r_y_total+1)))
aa= list(filter(lambda tupl: tupl[0]>=tupl[1] and (tupl[1]+1)>=(tupl[0]) ,aa))
dir_strides= einops.repeat(np.arange(true_num_dim),'a->(b a) 1',b=np.array(aa).shape[0]//true_num_dim)
print(f"dir_strides {dir_strides.shape} aa {np.array(aa).shape}")
np.concatenate([np.array(aa),dir_strides],axis=1)


dir_strides (6, 1) aa (6, 2)


array([[1, 0, 0],
       [1, 1, 1],
       [2, 1, 0],
       [2, 2, 1],
       [3, 2, 0],
       [3, 3, 1]])

In [30]:
bb_f=bb.flatten()
mask_num=4
main_index=3
mask_index=1
bb_f[main_index*mask_num+ mask_index]

13

In [38]:
def translate_in_axis(mask:jnp.ndarray, axis:int,is_forward:int,translation_val:int,mask_shape:Tuple[int]):
    mask= jnp.take(mask, indices=jnp.arange(translation_val*(1-is_forward),mask_shape[axis]-translation_val* is_forward),axis=axis )
    to_pad=np.array([[0,0],[0,0]])
    is_back=1-is_forward
    to_pad[axis,is_back]=translation_val
    res= jnp.pad(mask,to_pad)
    return res



Array([[ 0,  0,  0,  0],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]], dtype=int32)

In [40]:
def translate_in_axis_x_up(arr:jnp.ndarray):
    return jnp.roll(arr, 1, axis=0).at[0,:].set(0)
def translate_in_axis_x_down(arr:jnp.ndarray):
    return jnp.roll(arr, -1, axis=0).at[-1,:].set(0)
def translate_in_axis_y_up(arr:jnp.ndarray):
    return jnp.roll(arr, 1, axis=1).at[:,0].set(0)
def translate_in_axis_y_down(arr:jnp.ndarray):
    return jnp.roll(arr, 1, axis=1).at[:,-1].set(0)


In [13]:
x = jnp.arange(12).reshape(1,3, 4,1)
print(x[0,:,:,0])
print("\n ")
x_b=jax.lax.dynamic_slice(x,(0,1, 1,0), (1,2, 2,1))
print(x_b[0,:,:,0])
x_b.shape

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

 
[[ 5  6]
 [ 9 10]]


(1, 2, 2, 1)

In [4]:
# ConvDimensionNumbers(lhs_spec=(0, 3, 1, 2), rhs_spec=(3, 2, 0, 1), out_spec=(0, 3, 1, 2))

img_shape= (0,3, 1, 2)  
kernel_shape=(3, 2, 0, 1)
# kernel_shape=(32, 1, 5, 5)  
# img_shape= (70, 64, 32, 67)
# kernel= jnp.ones(kernel_shape)
# img= jnp.ones(img_shape)
kernel = jnp.zeros((5, 5,3, 10), dtype=jnp.float32)
img = jnp.zeros((60, 10, 11, 3), dtype=jnp.float32)
dn = lax.conv_dimension_numbers(img.shape,     # only ndim matters, not shape
                                kernel.shape,  # only ndim matters, not shape 
                                ('NHWC', 'HWIO', 'NHWC'))  # the important bit

res=jax.lax.conv_general_dilated(img, kernel,(1,1), 'SAME',
                              (2,1),  # lhs/image dilation
                               (1,1),  # rhs/kernel dilation
                               dn)    # dimension_numbers = lhs, rhs, out dimension permutation)))
res.shape

ValueError: String padding is not implemented for transposed convolution using this op. Please either exactly specify the required padding or use conv_transpose.

In [23]:
img_shape= (0,3, 1, 2)  
kernel_shape=(3, 2, 0, 1)
# kernel_shape=(32, 1, 5, 5)  
# img_shape= (70, 64, 32, 67)
# kernel= jnp.ones(kernel_shape)
# img= jnp.ones(img_shape)
kernel = jnp.zeros((5, 5,3, 10), dtype=jnp.float32)
img = jnp.zeros((60, 10, 11, 3), dtype=jnp.float32)
dn = lax.conv_dimension_numbers(img.shape,     # only ndim matters, not shape
                                kernel.shape,  # only ndim matters, not shape 
                                ('NHWC', 'HWIO', 'NHWC'))  # the important bit

res=jax.lax.conv_transpose(img, kernel,(2,1), 'SAME', (1,1),None)
                            #   (1,1),  # lhs/image dilation
                            #    (1,1),  # rhs/kernel dilation

# res= jnp.swapaxes(res,2,1)
                            #    dn)    # dimension_numbers = lhs, rhs, out dimension permutation)))
res.shape



(60, 20, 11, 10)

In [3]:
orbax_checkpointer=orbax.checkpoint.PyTreeCheckpointer()
raw_restored = orbax_checkpointer.restore('/workspaces/Jax_cuda_med/data/checkpoints/2023-06-12_06_21_11_143817/1755')
raw_restored['model']['params']
# state['params'].unfreeze()
# state['params']=raw_restored['model']['params']
# state['params']=freeze(state['params'])

{'Conv_trio_0': {'Conv_0': {'bias': array([[-0.00221546,  0.00133466,  0.00095959, -0.00218824,  0.00124864,
            0.00149952, -0.00059136,  0.00144733, -0.00042721,  0.00010629,
            0.0030192 , -0.00083613, -0.00348396, -0.00013437, -0.00225024,
           -0.0018575 ],
          [ 0.00040346,  0.00031311, -0.00037268,  0.00165125,  0.00042308,
           -0.00044971, -0.00093223, -0.00026955,  0.00031293, -0.00117358,
            0.00022168,  0.00074272, -0.00185966, -0.00289179, -0.00073372,
            0.00038402]], dtype=float32),
   'kernel': array([[[[[ 0.06260017,  0.29973814,  0.13903698,  0.20948595,
              -0.02901869, -0.25987002, -0.00434075, -0.18509352,
               0.07153301,  0.01981336, -0.00787897,  0.24355157,
               0.36041123, -0.17324458,  0.04364739, -0.22860353]],
   
            [[-0.13916749, -0.10466212,  0.32388458,  0.32949314,
              -0.05170148,  0.11824388, -0.30404723, -0.24549147,
              -0.07726976, -0.18

In [14]:
import itertools
all=jnp.array([[ 29, 285,   2]
 ,[ 29, 269  , 3]
 ,[ 30, 541,   0]
 ,[ 30, 542,   1]
 ,[ 30, 286,   2]
 ,[ 30, 270,   3]
 ,[ 31, 542,   0]
 ,[ 31, 543,   1]
 ,[ 31, 287,   2]
 ,[ 31, 271,   3]
 ,[ 32, 544,   1]
 ,[ 32, 288,   2]
 ,[ 32, 272,   3]
 ,[ 33, 544,   0]
 ,[ 33, 545,   1]
 ,[ 33, 289,   2]
 ,[ 33, 273,   3]
 ,[ 34, 545,   0]
 ,[ 34, 546,   1]
 ,[ 34, 290,   2]])

grouped_edges_dict = {k: list(v) for k, v in itertools.groupby(np.array(all), key=lambda x: x[0])}

sorted_keys=sorted(grouped_edges_dict.keys())
def get_uniform_shape(grouped_edges_dict,key):
    stacked=jnp.stack(grouped_edges_dict[key])
    shapee=stacked.shape
    print(f"shapee {shapee}")
    return jnp.pad(stacked,((0,4-shapee[0]),(0,0)))


get_uniform_shape(grouped_edges_dict,sorted_keys[3])

shapee (3, 3)


Array([[ 32, 544,   1],
       [ 32, 288,   2],
       [ 32, 272,   3],
       [  0,   0,   0]], dtype=int32)

In [2]:
def for_v_map_set(point,to_modif,new_val):
    return jax.lax.dynamic_update_slice(to_modif, jnp.array([[new_val]]), point)

point=jnp.array([1,1])
to_modif= jnp.zeros((5,5))
new_val=1.0

for_v_map_set(point,to_modif,new_val)

Array([[0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [4]:
a= jnp.array([[0,1,1],[1,2,0],[1,1,1] ])
jnp.where(a[2]==1,a)

TypeError: where requires ndarray or scalar arguments, got <class 'NoneType'> at position 2.

In [6]:
from einops import pack, unpack

h, w = 4, 4
# image_rgb is 3-dimensional (h, w, 3) and depth is 2-dimensional (h, w)
image_rgb = np.random.random([h, w, 3,7])
image_depth = np.random.random([h, w,7])
# but we can stack them
image_rgbd, ps = pack([image_rgb, image_depth], 'h w * r')
print(f"image_rgbd {image_rgbd.shape}")
unpacked_rgb, unpacked_depth = unpack(image_rgbd, ps, 'h w * r')
unpacked_rgb.shape, unpacked_depth.shape


unpack(image_rgbd, ps, 'h w * r')

image_rgbd (4, 4, 4, 7)


[array([[[[0.52175407, 0.60162895, 0.81111374, 0.79985516, 0.69305018,
           0.79323732, 0.25707958],
          [0.0832438 , 0.33833896, 0.31236225, 0.54743057, 0.7141946 ,
           0.0451901 , 0.16437969],
          [0.00210303, 0.74843234, 0.52454854, 0.03151455, 0.69339701,
           0.99227028, 0.24550495]],
 
         [[0.8837042 , 0.91976362, 0.03409993, 0.84705071, 0.07793035,
           0.72255673, 0.63030197],
          [0.95528161, 0.70522354, 0.1068413 , 0.08777508, 0.55864409,
           0.13585931, 0.46511545],
          [0.9649139 , 0.337316  , 0.95418335, 0.51327874, 0.68010127,
           0.11564217, 0.31546615]],
 
         [[0.94058876, 0.53559073, 0.92528562, 0.11220068, 0.71819945,
           0.99016471, 0.25748775],
          [0.53795112, 0.68069593, 0.44202157, 0.44643175, 0.47438319,
           0.33336585, 0.80795089],
          [0.5861211 , 0.80238431, 0.87029804, 0.63965597, 0.28021181,
           0.16880913, 0.80567788]],
 
         [[0.64919673, 0.920

In [5]:
def get_initial_supervoxel_masks(orig_grid_shape,shift_x,shift_y,mask_num):
    """
    on the basis of the present shifts we will initialize the masks
    ids of the supervoxels here are implicit based on which mask and what location we are talking about
    """
    initt=np.zeros(orig_grid_shape)
    initt[shift_x::2,shift_y::2,mask_num]=1
    return initt

orig_grid_shape=(8,8,4)
initial_masks= jnp.stack([
            get_initial_supervoxel_masks(orig_grid_shape,0,0,0),
            get_initial_supervoxel_masks(orig_grid_shape,1,0,1),
            get_initial_supervoxel_masks(orig_grid_shape,0,1,2),
            get_initial_supervoxel_masks(orig_grid_shape,1,1,3)
                ],axis=0)
initial_masks=jnp.sum(initial_masks,axis=0)
print(initial_masks[:,:,0])
print(initial_masks[:,:,1])
print(initial_masks[:,:,2])
print(initial_masks[:,:,3])

[[1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 1. 0.]]
[[0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1.]]


In [7]:
a=jnp.ones((5,7,4))
index=0
b=jax.lax.dynamic_index_in_dim(a, index, axis=-1, keepdims=True)
b.shape

(5, 7, 1)